# Loading McDonalds Dataset

In [29]:
import pandas as pd
import matplotlib.pyplot as plt

print('------ Data Head ------')
try:
    data = pd.read_csv('McDonald_s_Reviews.csv', encoding='latin1')
    print(data.head())
except Exception as e:
    print("Error reading file with latin1 encoding:", e)

print('\n')
print('------ Data Info ------')
print(data.info())
print('\n')
print('------ Descriptive Statistics ------')
print(data.describe())


------ Data Head ------
   reviewer_id  store_name              category  \
0            1  McDonald's  Fast food restaurant   
1            2  McDonald's  Fast food restaurant   
2            3  McDonald's  Fast food restaurant   
3            4  McDonald's  Fast food restaurant   
4            5  McDonald's  Fast food restaurant   

                                       store_address  latitude   longitude  \
0  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
1  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
2  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
3  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   
4  13749 US-183 Hwy, Austin, TX 78750, United States  30.460718 -97.792874   

  rating_count   review_time  \
0        1,240  3 months ago   
1        1,240    5 days ago   
2        1,240    5 days ago   
3        1,240   a month ago   
4        1,240  2 months ago   

 

#### Data Cleaning

In [32]:
# Trim spaces from column names
data.columns = data.columns.str.strip()

# Check the columns to ensure the names are correct
print("Corrected Columns in DataFrame:", data.columns)

# Proceed with filling missing values
data['latitude'].fillna(data['latitude'].median(), inplace=True)
data['longitude'].fillna(data['longitude'].median(), inplace=True)


Corrected Columns in DataFrame: Index(['reviewer_id', 'store_name', 'category', 'store_address', 'latitude',
       'longitude', 'rating_count', 'review_time', 'review', 'rating'],
      dtype='object')


## Sentiment Analysis

In [48]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# Initialize the Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Example: Applying sentiment analysis to the first 5 reviews
data['review'].head().apply(lambda x: sia.polarity_scores(x))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kanishkaw/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


0    {'neg': 0.027, 'neu': 0.879, 'pos': 0.094, 'co...
1    {'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'comp...
2    {'neg': 0.051, 'neu': 0.949, 'pos': 0.0, 'comp...
3    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4    {'neg': 0.143, 'neu': 0.857, 'pos': 0.0, 'comp...
Name: review, dtype: object

In [50]:
# Check the data type of 'rating'
print("Data type of 'rating':", data['rating'].dtype)
print("First few entries in 'rating':", data['rating'].head())

# If 'rating' is not an integer, convert it
if data['rating'].dtype == object:
    data['rating'] = data['rating'].str.extract('(\d+)').astype(int)

# Confirm the conversion if it was necessary
print("Updated data type of 'rating':", data['rating'].dtype)
print(data['rating'].head())

# Correlation between compound sentiment scores and numerical ratings
print("Correlation between compound scores and ratings:")
print(data[['compound', 'rating']].corr())

# Visualization of sentiment distribution
data['compound'].hist(bins=20)
plt.title('Distribution of Sentiment Scores')
plt.xlabel('Compound Sentiment Score')
plt.ylabel('Frequency')
plt.show()



Data type of 'rating': int64
First few entries in 'rating': 0    1
1    4
2    1
3    5
4    1
Name: rating, dtype: int64
Updated data type of 'rating': int64
0    1
1    4
2    1
3    5
4    1
Name: rating, dtype: int64
Correlation between compound scores and ratings:


KeyError: "['compound'] not in index"